In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from rdkit import RDLogger
# Disable RDKit error logging
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

In [2]:
# IMS data from Predicting differential ion mobility behaviour in silico using machine learning 
# --authors: Christian Ieritano, J. Larry Campbella, and W. Scott Hopkins
file_path = '/home/cmdunham/ChemicalDataGeneration/data/predicting_ion_mobility_iertiano_et_al_data.csv'
data = pd.read_csv(file_path)

file_path = '/home/cmdunham/ChemicalDataGeneration/data/predicting_ion_mobility_iertiano_et_al_inchikeys.csv'
inchikey_df = pd.read_csv(file_path)

# Importing the MoNA database of mass spec data, just for use as database to go from compound names to smiles and inchikeys. I haven't found a better way to do this efficiently.
file_path = '/home/cmdunham/ChemicalDataGeneration/data/MoNA_data/MoNA-export-GC-MS_Spectra.sdf'
suppl = Chem.SDMolSupplier(file_path)

In [3]:
data.head()

,Compound,m/z,SV 1500,SV 2000,SV 2500,SV 3000,SV 3250,SV 3500,SV 3750,SV 4000,Boltzmann-weighted CCS
0,5-Cl-MQOH,194.04,0.1,0.1,0.3,0.5,0.91,1.6,2.57,3.8,138.66
1,5-CN-MQOH,185.07,0.1,0.1,-0.2,-0.3,-0.14,0.2,0.77,1.6,132.21
2,5-F-MQOH,178.07,-0.2,-0.4,-0.5,-0.3,0.08,0.7,1.60,2.8,127.47
3,5-Me-MQOH,174.09,-0.2,-0.2,0.0,0.8,1.47,2.4,3.65,5.3,129.45
4,5-NO2-MQOH,205.06,0.2,-0.1,0.0,-0.4,-0.34,0.0,0.59,1.4,137.66


In [4]:
compounds = list(data['Compound'])
print(f'There are {len(set(compounds))} compounds in the ion mobility dataset.')

There are 409 compounds in the ion mobility dataset.


In [5]:
print(f"There are {len(data['Compound'])} compounds in the ion mobility dataset and {len(inchikey_df['Compound'])} compounds in the corresponding inchikey file.")
print(f"Of those, {len(set(inchikey_df['Compound'])& set(data['Compound']))} overlap between the two files.")

There are 409 compounds in the ion mobility dataset and 409 compounds in the corresponding inchikey file.
Of those, 406 overlap between the two files.


In [25]:
print(set(inchikey_df['Compound']) - set(data['Compound']))
print(set(data['Compound']) - set(inchikey_df['Compound']))

{'(+)-Propoxyphene', 'Norsertraline', 'Noroxycodone'}
{'Norsertraline HCL', 'Noroxycodone HCL', 'Propoxyphene'}


In [27]:
# changing the compound names in the inchikey df so they correspond to the names in the ims df
inchikey_df.loc[inchikey_df['Compound'] == '(+)-Propoxyphene'] = 'Propoxyphene'
inchikey_df.loc[inchikey_df['Compound'] == 'Norsertraline'] = 'Norsertraline HCL'
inchikey_df.loc[inchikey_df['Compound'] == 'Noroxycodone'] = 'Noroxycodone HCL'
print(set(data['Compound']) - set(inchikey_df['Compound']))

set()


In [28]:
# creating a dict of compound names and inchikeys for all compounds in the ion mobility dataset
compound_inchikey_dict = dict(zip(inchikey_df['Compound'], inchikey_df['InChIKey']))

Chemception needs SMILES to generate embeddings. The ion mobility dataset has InCHIKey but no SMILES. Since the MoNA dataset has both I'm using that as a way to go from one to the other. I haven't found a way to do this more efficiently, the other option I'm aware of is manually looking up each compound's InCHIKey somewhere like NIST Webbook, which is VERY time consuming and prone to human error (namely me copying things incorrectly). 

MoNA samples contain SMILES in the comment, not as a property, so the SMILES must be pulled out of the comment to use.

In [12]:
def format_smiles(mol):
    """
    Extracts the SMILES string from the COMMENT property of an rdkit molecule.

    Parameters:
    mol: An rdkit molecule.

    Returns:
    str or None: The SMILES string if found, otherwise None.
    """
    
    comment = mol.GetProp('COMMENT').split('\n')
    smiles = None
    for line in comment:
        if line.split('=')[0] == 'SMILES':
            smiles = line.split('SMILES=')[1]

    return smiles

In [29]:
# get SMILES for any compounds in the MoNA database that are also in the Ion Mobiity dataset and create a dictionary with inchikeys as keys and SMILES as values
overlapping_mol_inchikey_smiles_dict = {}

for mol in suppl:
    if mol is not None:
        try:
            inchikey = mol.GetProp('INCHIKEY')

        # ignore any mols without inchikeys
        except:
            continue
        
        # if this compound is in the ion mobility dataset
        if inchikey in compound_inchikey_dict.values():
            # if this compound's smiles hasn't been recorded yet
            if not inchikey in overlapping_mol_inchikey_smiles_dict.keys():
                smiles = format_smiles(mol)
                if smiles:
                    overlapping_mol_inchikey_smiles_dict[inchikey] = smiles


In [30]:
# Get a list of compounds from the ion mobility dataset that also appear in the MoNA dataset.
compounds_in_both_datasets = []
for mol in suppl:
    if mol is not None:
        if mol.GetProp('NAME') in compounds:
            compounds_in_both_datasets.append(mol.GetProp('NAME'))

We see that there are more InChIKeys in common between the datasets than there are compounds names. This is likely due to the fact that InChIKeys are unique identifiers while the same chemical might can have multiple compound names, meaning that it could be listed one way in one dataset and another way in another.

In [31]:
print(f'There are {len(set(compounds_in_both_datasets))} compound names from the Ion Mobility dataset that appear in the MoNA dataset compared to {len(overlapping_mol_inchikey_smiles_dict)} inchikeys that appear in both datasets.')

There are 48 compound names from the Ion Mobility dataset that appear in the MoNA dataset compared to 76 inchikeys that appear in both datasets.


In [32]:
# make lists of inchikeys and SMILES to add as columns to the IMS dataframe
inchikeys = []
smiles = []

for mol in compounds:
    inchikey = compound_inchikey_dict[mol]
    
    # look mol up in dict and append corresponding inchikey to list
    inchikeys.append(inchikey)
    
    # if inchikey is in the dict of mols with SMILES from MoNA, get the SMILES and append to list
    if inchikey in overlapping_mol_inchikey_smiles_dict.keys():
        smiles.append(overlapping_mol_inchikey_smiles_dict[inchikey])
    else:
        smiles.append(None)

In [33]:
# Double checking that the recorded SMILES/InChIKey pairs correspond
total_errors = 0
for i, smile in enumerate(smiles):
    if smile:
        if not overlapping_mol_inchikey_smiles_dict[inchikeys[i]] == smiles[i]:
            print(f'SMILES/InChIKey pair at {i} does not match what is specified in the lookup dictionary.')
            total_errors+=1
if not total_errors:
    print('All SMILES/InChIKey pairs match what is specified in the lookup dictionary.')

All SMILES/InChIKey pairs match what is specified in the lookup dictionary.


Adding columns for InChIKey and SMILES. InChIKey will be needed for identification, SMILES for retrieving Chemception embeddings.

In [34]:
ims_data = data.copy()
ims_data['SMILES'] = smiles
ims_data['InChIKey'] = inchikeys
ims_data = ims_data.dropna(subset=['SMILES']).reset_index(drop=True)
# ims_data.drop(columns=['Compound'], inplace=True)
ims_data.head()

,Compound,m/z,SV 1500,SV 2000,SV 2500,SV 3000,SV 3250,SV 3500,SV 3750,SV 4000,Boltzmann-weighted CCS,SMILES,InChIKey
0,6-methyl-2-MQ,158.10,0.20,0.20,0.60,1.90,3.03,4.50,6.32,8.50,126.14,Cc(c2)cc(c1)c(c2)nc(C)c1,JJPSZKIOGBRMHK-UHFFFAOYSA-N
1,MQOH,160.08,-0.40,-0.60,-0.70,-0.40,0.06,0.80,1.89,3.40,125.75,Cc(c1)nc(c2)c(ccc2)c1,SMUQFGGVLNAIOZ-UHFFFAOYSA-N
2,Ametryn,228.13,0.44,1.15,2.38,4.04,5.07,6.37,7.67,9.29,151.92,CCNc(n1)nc(SC)nc(NC(C)C)1,RQVYBGPQFYCBGX-UHFFFAOYSA-N
3,Azoxystrobin,404.13,0.66,1.65,3.14,5.46,7.03,8.77,10.78,13.03,203.24,CO/C=C(\C1=CC=CC=C1OC2=NC=NC(=C2)OC3=CC=CC=C3C...,WFDXOXNFNRHQEC-GHRIWEEISA-N
4,Dimethoate,230.01,0.48,1.32,2.71,4.80,6.18,7.88,9.57,11.49,140.98,CNC(=O)CSP(=S)(OC)OC,MCWXGJITAZMZEV-UHFFFAOYSA-N
